<a href="https://colab.research.google.com/github/alektebel/alektebel.github.io/blob/main/Diego_Rodr%C3%ADguez_AG1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actividad guiada 1

Nombre: Diego Rodríguez Atencia

Actividad guiada 1 de algoritmos de optimización

url_gh_AG1 = https://github.com/alektebel/03MAIR--Algoritmos-de-optimizacion--2024/tree/main/AG1

colab = https://colab.research.google.com/drive/1Yjm-akPK1MqL3Q_U30S0c1qnHGTlK5EI?authuser=2#scrollTo=q1aeel1wHbDp

### Decorador para medir el tiempo de ejecución

In [ ]:
def medir_tiempo(func):
  def wrapper(*args, **kwargs):
    import time
    inicio = time.time()
    resultado = func(*args, **kwargs)
    tiempo = (time.time() - inicio) *1000000
    print(f"Tiempo de ejecución: {tiempo:.2f} microseconds")
    return resultado
  return wrapper



Algoritmo de euclides

In [ ]:
@medir_tiempo
def euclides(a, b):
  if b < a:
    b, a = a, b
  while(b):
    a, b = b, a % b
  return a

In [ ]:
test_time = [(105000, 400000), (20000230, 10000650), (1870000000, 1000654000)]


for test in test_time:
  print(f"Test: {test}")
  euclides(*test)


Test: (105000, 400000)
Tiempo de ejecución: 3.34 microseconds
Test: (20000230, 10000650)
Tiempo de ejecución: 1.91 microseconds
Test: (1870000000, 1000654000)
Tiempo de ejecución: 3.10 microseconds


Algoritmo de heron

In [ ]:
@medir_tiempo
def heron(x0, p, err, tol = 0.0001, max_iter = 1000):
  iter = 0
  while(abs(x0**2-p) > err):
    x_temp = x0
    x0 = (x0 + p / x0) / 2
    if abs(x_temp - x0) < tol:
      print("Se alcanzó la tolerancia")
      return x_temp
    iter += 1
    if iter > max_iter:
      print("Se alcanzó el número máximo de iteraciones")
      return x0
  return x0

In [ ]:
tests = [2, 3, 4, 5, 6]
tol = 0.001

import numpy as np

for test in tests:
  print(f"Test: {test}")
  print('La raiz de '+str(test) + ' -> ' +str(heron(1, test, tol)))
  print('La raiz real es: ', np.sqrt(test))

Test: 2
Tiempo de ejecución: 32.66 microseconds
La raiz de 2 -> 1.4142156862745097
La raiz real es:  1.4142135623730951
Test: 3
Tiempo de ejecución: 10.25 microseconds
La raiz de 3 -> 1.7321428571428572
La raiz real es:  1.7320508075688772
Test: 4
Tiempo de ejecución: 5.96 microseconds
La raiz de 4 -> 2.0000000929222947
La raiz real es:  2.0
Test: 5
Tiempo de ejecución: 5.72 microseconds
La raiz de 5 -> 2.2360688956433634
La raiz real es:  2.23606797749979
Test: 6
Tiempo de ejecución: 5.01 microseconds
La raiz de 6 -> 2.4494943716069653
La raiz real es:  2.449489742783178


## Problema de autobuses


Una pequeña ciudad estudia introducir un sistema de transporte urbano de
autobuses. Nos encargan estudiar el número mínimo de autobuses para
cubrir la demanda. Se ha realizado un estudio para estimar el número
mínimo de autobuses por franja horaria. Lógicamente este número varia
dependiendo de la hora del día. Se observa que es posible dividir la franja
horaria de 24h en tramos de 4 horas en los queda determinado el número
de autobuses que se necesitan. Debido a la normativa cada autobús debe
circular 8 horas como máximo y seguidas en cada jornada de 24h

### IMPLEMENTACION
Si representamos el inicio de los turnos de los buses como un array de tantos elementos como tramos hay, tenemos un vector $x_0, x_1, \cdots, x_n$ cuya suma es el número de buses que tenemos. Dado esto, queremos minimizar el número total de buses que utilizamos
$$
   \sum_t x_t
$$
Bajo las condiciones de:
$$
x_t+x_{t+1} \leq r_t
$$
Donde $r_t$ son los autobuses requeridos en la franja de tiempo t.



#### SOLUCION INGENUA
solucion_ingenua = [8, 2, 5, 7, 0, 9]
vs
demanda =          [8, 10, 7, 12, 6, 9]

Esta es una solución ingenua, en la que se necesitan 31 autobuses. La solución minima por tanto deberá ser menor o igual que esta, es decir, que no puede ser mayor.


Queremos recorrer todas las posible combinaciones de horarios para n buses, asignados a m tramos, donde n > m. Esto es el problema de dividir estrellas entre casillas, en el que tenemos m casillas y queremos asignar n estrellas entre las casillas. Para ello, lo tenemos en cuenta como si hubiesen n + m -1 posiciones: Por ejemplo, para n = 5 y m = 3
$$*****|||$$
Hay en total 5 + 3 - 1 posibles posiciones, que en total son 9. Queremos encontrar todas las formas de organizar las estrellas y barras sin contar las permutaciones triviales (| con | o * con *). Entonces, lo que queremos es contar todos los subgrupos de m -1 elementos (fijandonos en las barras) sobre todos estos elementos, es decir:
$$\binom{m + n - 1}{m -1}$$

Establecemos un primer rango de búsqueda entre 30 y 32:

In [3]:
from itertools import permutations
import math

def generar_soluciones(n, m):
  if m == 1:  # Caso base, cuando sólo quede un tramo, el resto de buses irá aquí.
    return [[n]]
  combinations = []
  for i in range(n + 1):  # Añadimos i buses al primer tramo
      for rest in generar_soluciones(n - i, m - 1):  # Distribuimos los n-i buses restantes en las casillas que quedarán de forma recursiva, y cuando quede sólo un tramo, se pondrán todos los buses
      #que queden, por el caso base de arriba.
          combinations.append([i] + rest)
  return combinations

def solucion_valida(solution, req_buses):
    """
    Comprueba si una solución es válida
    """
    #Comprobamos el turno de noche
    if solution[0]+ solution[-1] < req_buses[0]:
      return False
    for i in range(len(req_buses)-1):
        if solution[i] + solution[i+1] < req_buses[i+1]:#Los buses deben estar de tal forma que se ajusten a la correspondiente demanda
            return False
    return True

def encontrar_solucion_brute_force(demanda, lb_buses = 15, up_buses = 30):
    """
    Encuentra alguna solución posible utilizando fuerza bruta
    """
    min_buses = float('inf')
    mejor_solucion = None

    #buscamos soluciones con número de buses entre lower_bound_buses y upper_bound_buses
    for num_buses in range(lb_buses, up_buses+1):
      print('Buscando solución para ', num_buses, ' buses')
      for sol in generar_soluciones(num_buses, len(demanda)):
        if solucion_valida(sol, demanda):
          if num_buses < min_buses:
            min_buses = num_buses
            mejor_solucion = sol
            print('La solucion es: ', sol)
            break
      if min_buses == float('inf'):
        print('Ninguna solución para ', num_buses, ' buses.')
    return min_buses, mejor_solucion


if __name__ == "__main__":
    demand = [4, 8, 10, 7, 12, 4]

    min_buses, solucion = encontrar_solucion_brute_force(demand, 24, 26)


Buscando solución para  24  buses
Ninguna solución para  24  buses.
Buscando solución para  25  buses
Ninguna solución para  25  buses.
Buscando solución para  26  buses
La solucion es:  [0, 8, 2, 5, 7, 4]


Como hemos podido ver, parece que se encuentra solución para 31 buses, que es el número equivalente de buses a encontrar para el ejemplo ingenuo. La complejidad de esta búsqueda por fuerza bruta se encuentra en $$O(\frac{m+n-1!}{m-1! n!})  = O(m^n)$$ donde m es el número de tramos, y n es el mayor número de buses para el que realizamos la búsqueda. puesto que tenemos que comprobar todas las combinaciones en las que podemos desglosar un número n (el número de autobuses) entre todos los posibles horarios de tramo en 24 horas (m-1). Se ha buscado esta solución suponiendo que no hay turnos de 4 horas, pero dado que según el enunciado, los turnos tienen que ser seguidos y como máximo de 8 horas (con lo que suponemos que todos los turnos deberían ser de 8 horas para optimizar al máximo).